In [ ]:
import cv2
import numpy as np
import os
import shutil  # 用于复制文件

# 输入和输出文件夹路径
input_folder_path = '/data1/xjlDatasets/LIDC/manifest-1600709154662/data_AMISDM/training'
output_folder_path = '/data1/xjlDatasets/LIDC/manifest-1600709154662/data_AMISDM_Sober/training'

# 确保输出文件夹路径存在
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# 获取所有子文件夹（每个数据集）
for root, dirs, files in os.walk(input_folder_path):
    # 处理每个子文件夹
    for dir_name in dirs:
        folder_path = os.path.join(root, dir_name)
        output_subfolder = os.path.join(output_folder_path, dir_name)
        
        # 确保输出子文件夹存在
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)
        
        # 读取标签图像
        label_files = ['label0_.png', 'label1_.png', 'label2_.png', 'label3_.png']
        labels = []

        # 加载图像并检查是否成功加载
        for file in label_files:
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"图像加载失败: {img_path}")
            else:
                labels.append(img)

        # 检查是否所有图像都成功加载
        if len(labels) != len(label_files):
            print(f"加载失败的标签图像数量: {len(label_files) - len(labels)}")
            continue
        else:
            print(f"成功加载 {len(labels)} 张标签图像, 当前处理文件夹: {folder_path}")

        # 读取原始图像（假设原始图像为 'image_序号.jpg'）
        image_path = os.path.join(folder_path, f'image_{dir_name}.jpg')  # 根据文件夹名来确定文件名
        image = cv2.imread(image_path)
        if image is None:
            print(f"原始图像加载失败: {image_path}")
            continue

        # 提取标签区域（白色区域）
        def extract_label_region(label):
            _, label_region = cv2.threshold(label, 1, 255, cv2.THRESH_BINARY)
            return label_region

        # 提取边界区域（通过Sobel算子）
        def compute_boundary_probability(label):
            grad_x = cv2.Sobel(label, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(label, cv2.CV_64F, 0, 1, ksize=3)
            magnitude = cv2.magnitude(grad_x, grad_y)
            # 归一化边界强度到[0, 1]范围，将边缘位置的强度归为接近0
            boundary_prob = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX)
            return boundary_prob

        # 为每个标签图像提取标签区域
        label_regions = [extract_label_region(label) for label in labels]

        # 计算每个标签图像的边界区域
        boundary_probs = [compute_boundary_probability(label_region) for label_region in label_regions]

        # 生成标签的概率图，调整边界的权重
        def generate_probability_map(label_regions, boundary_probs, labels, boundary_weight=1.2, center_weight=1, unlabeled_weight=0.3):
            probability_map = np.zeros_like(label_regions[0], dtype=np.float32)

            for label_region, boundary_prob, label in zip(label_regions, boundary_probs, labels):
                # 对标签区域，赋值为接近1
                probability_map += label_region / 255.0  # 标签区域设置为1（最大概率）

                # 对边界区域，概率值按边界强度进行调整，边界区域概率接近0
                probability_map += boundary_weight * boundary_prob * (1 - label_region / 255.0)  # 边界区域设置为接近0的概率

                # 中间区域增强权重（保持中心区域较高的概率）
                probability_map += center_weight * (1 - boundary_prob) * (label_region / 255.0)  # 中间区域保持较高的概率

                # 拉低未标注区域的概率值
                unlabeled_region = (label == 0)  # 判断该医生未标注的区域（黑色区域）
                probability_map[unlabeled_region] *= unlabeled_weight  # 对未标注区域的概率进行拉低处理

            return probability_map / len(label_regions)  # 对所有标签进行平均

        # 计算融合后的概率图
        fused_probability = generate_probability_map(label_regions, boundary_probs, labels)

        # 保存融合后的概率图像（在[0, 1]范围内）
        output_fused_path = os.path.join(output_subfolder, 'label_Sober.png')  # 保存路径
        cv2.imwrite(output_fused_path, fused_probability * 255)  # 保存为图像，乘以255以确保是可视化的概率图

        # 复制原始图像到新文件夹
        output_image_path = os.path.join(output_subfolder, f'image_{dir_name}.jpg')  # 保存路径
        shutil.copy(image_path, output_image_path)  # 复制原始图像

        # 输出保存路径
        print(f"最终融合的概率图像已保存到：{output_fused_path}")
        print(f"原始图像已保存到：{output_image_path}")


In [ ]:
import os
import cv2
import torch
import numpy as np
import shutil

# 输入和输出文件夹路径
input_folder_path = '/data1/xjlDatasets/LIDC/manifest-1600709154662/data_AMISDM/testing'
output_folder_path = '/data1/xjlDatasets/LIDC/manifest-1600709154662/data_AMISDM_shang/testing'

os.makedirs(output_folder_path, exist_ok=True)

for dir_name in os.listdir(input_folder_path):
    input_subfolder = os.path.join(input_folder_path, dir_name)
    output_subfolder = os.path.join(output_folder_path, dir_name)
    os.makedirs(output_subfolder, exist_ok=True)

    label_files = ['label0_.png', 'label1_.png', 'label2_.png', 'label3_.png']
    labels = []

    for file in label_files:
        img_path = os.path.join(input_subfolder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = img.astype(np.float32) / 255.0
        labels.append(img)

    if len(labels) == 0:
        continue

    labels_tensor = torch.tensor(np.stack(labels, axis=0))

    # 计算标注医生的数量
    num_doctors_labeling = labels_tensor.sum(dim=0)

    # 初始概率设置
    P_mean = num_doctors_labeling / 4.0

    # 计算熵
    entropy_eps = 1e-6
    entropy = -(P_mean * torch.log(P_mean + entropy_eps) + (1 - P_mean) * torch.log(1 - P_mean + entropy_eps))
    entropy_norm = (entropy - entropy.min()) / (entropy.max() - entropy.min() + entropy_eps)

    # 根据熵值动态调整
    adjustment_strength = 0.1
    adjustment = torch.where(entropy_norm > entropy_norm.mean(), adjustment_strength * entropy_norm, -adjustment_strength * entropy_norm)

    # 明确标注情况概率不变（0或1）
    adjustment[num_doctors_labeling == 4] = 0
    adjustment[num_doctors_labeling == 0] = 0

    # 调整概率值
    fused_probability = P_mean + adjustment
    fused_probability = torch.clamp(fused_probability, 0, 1).numpy()

    # 保存融合后的概率图像
    output_fused_path = os.path.join(output_subfolder, 'label_Shang.png')
    cv2.imwrite(output_fused_path, (fused_probability * 255).astype(np.uint8))

    # 复制原始图像到新文件夹
    image_path = os.path.join(input_subfolder, f'image_{dir_name}.jpg')
    output_image_path = os.path.join(output_subfolder, f'image_{dir_name}.jpg')
    if os.path.exists(image_path):
        shutil.copy(image_path, output_image_path)

    print(f"融合标签和原始图像已保存至: {output_subfolder}")
